In [2]:
from get_weather import get_weather_data

In [3]:
import pandas as pd

In [4]:
import folium

In [4]:
x_data = get_weather_data("-19.7502", "-43.9194", "2024-04-09", "2024-04-09")

In [5]:
weather_df = pd.DataFrame({key: value for key, value in x_data["hourly"]. items()})

In [6]:
weather_df["time"] = pd.to_datetime(weather_df["time"])

In [7]:
filtered_hour = weather_df[weather_df["time"].dt.hour == 12]

In [7]:
cities = [
    {"name": "Uberlândia", "latitude": "-18.9186", "longitude": "-48.2772"},
    {"name": "Uberaba", "latitude": "-19.7472", "longitude": "-47.9381"},
    {"name": "Araguari", "latitude": "-18.6456", "longitude": "-48.1934"},
    {"name": "Ituiutaba", "latitude": "-18.9730", "longitude": "-49.4633"},
    {"name": "Patos de Minas", "latitude": "-18.5793", "longitude": "-46.5142"},
    {"name": "Frutal", "latitude": "-20.0242", "longitude": "-48.9350"},
    {"name": "Monte Carmelo", "latitude": "-18.7302", "longitude": "-47.4916"},
    {"name": "Araxá", "latitude": "-19.5902", "longitude": "-46.9438"}
]

In [10]:
start_date = "2024-04-09"
end_date = "2024-04-09"

city_weather_data = []

for city in cities:
    city_data = get_weather_data(city["latitude"], city["longitude"], start_date, end_date)
    
    city_df = pd.DataFrame({key: value for key, value in city_data["hourly"].items()})
    city_df["time"] = pd.to_datetime(city_df["time"])
    
    filtered_data = city_df[city_df["time"].dt.hour == 12].copy()
    
    filtered_data["city"] = city["name"]
    filtered_data["latitude"] = city["latitude"]
    filtered_data["longitude"] = city["longitude"]
    
    city_weather_data.append(filtered_data)

final_df = pd.concat(city_weather_data, ignore_index=True)

In [11]:
print(final_df)

                 time  wind_speed_10m            city  latitude longitude
0 2024-04-09 12:00:00             9.4      Uberlândia  -18.9186  -48.2772
1 2024-04-09 12:00:00            11.4         Uberaba  -19.7472  -47.9381
2 2024-04-09 12:00:00             5.6        Araguari  -18.6456  -48.1934
3 2024-04-09 12:00:00             8.2       Ituiutaba  -18.9730  -49.4633
4 2024-04-09 12:00:00             3.1  Patos de Minas  -18.5793  -46.5142
5 2024-04-09 12:00:00             7.9          Frutal  -20.0242  -48.9350
6 2024-04-09 12:00:00             5.9   Monte Carmelo  -18.7302  -47.4916
7 2024-04-09 12:00:00             7.4           Araxá  -19.5902  -46.9438


In [18]:
# Create a Folium map centered at the average latitude and longitude
map_center = [final_df["latitude"].astype(float).mean(), final_df["longitude"].astype(float).mean()]
folium_map = folium.Map(location=map_center, zoom_start=2)

# Add markers for each city
for _, row in final_df.iterrows():
    folium.CircleMarker(
        location=[float(row["latitude"]), float(row["longitude"])],
        radius=row["wind_speed_10m"] / 2,  # Adjust the size based on temperature
        color="blue",
        fill=False,
        fill_color="blue",
        fill_opacity=0.5,
        tooltip=row["wind_speed_10m"],
    ).add_to(folium_map)

# Display the map
folium_map